In [25]:
import os
import sys

import pandas as pd

In [26]:
table_dir = '/Users/eee/python/jra_ml/data/table'
target_file = 'horse_result.csv'

output_dir = '/Users/eee/python/jra_ml/data/input'
output_file = 'input.csv'

table_path = os.path.join(table_dir, target_file)
output_path = os.path.join(output_dir, output_file)

In [27]:
df = pd.read_csv(table_path)
df.shape

/var/folders/8c/ctv36bkx7pv_b47nj404bc480000gn/T/ipykernel_87236/4198753748.py:1: DtypeWarning: Columns (17,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(table_path)


(2570740, 29)

# 日本レースを抽出

In [28]:
place_urban = ['札幌', '函館', '福島', '新潟', '東京', '中山', '中京', '京都', '阪神', '小倉']
place_rural = ['帯広', '門別', '盛岡', '水沢', '浦和', '船橋', '大井', '川崎', '金沢', '笠松', '名古屋', '園田', '姫路', '高知', '佐賀']
#df = df[df['開催'].isin(place_urban + place_rural)]
re_patt = ''
for e in place_urban + place_rural:
    re_patt += e + '|'
re_patt = re_patt.rstrip('|')

df = df[df['開催'].str.contains(re_patt)]


# 欠損値処理

In [29]:
df = df.dropna(subset=['着順', '頭数', '騎手'])
df = df[df['着順'] != '失']
df = df[df['着順'] != '取']
df = df[df['着順'] != '中']
df = df[df['着順'] != '除']
df = df[df['頭数'] != 0]

df.head()

,horse_id,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,...,着差,ﾀｲﾑ指数,通過,ペース,上り,馬体重,厩舎ｺﾒﾝﾄ,備考,勝ち馬(2着馬),賞金
0,2020105571,2022/09/24,4中山6,曇,9.0,カンナS(OP),NaN,10.0,3.0,3.0,...,0.3,**,3-3,33.8-35.6,35.6,466(+2),NaN,NaN,ウメムスビ,403.3
1,2020105571,2022/09/03,2札幌7,晴,1.0,2歳未勝利,NaN,14.0,7.0,12.0,...,-0.5,**,1-1,35.0-35.7,35.7,464(-2),NaN,NaN,(コパカバーナビーチ),520.0
2,2020105571,2022/08/27,2札幌5,曇,2.0,2歳未勝利,NaN,8.0,4.0,4.0,...,0.1,**,3-3,34.5-35.9,35.5,466(+2),NaN,NaN,バロットボックス,210.0
3,2020105571,2022/08/06,1札幌5,晴,5.0,2歳新馬,NaN,8.0,3.0,3.0,...,0.8,**,5-5,36.1-34.0,34.4,464(0),NaN,NaN,メイクアスナッチ,70.0
4,2020101863,2022/09/25,4中山7,晴,2.0,2歳未勝利,NaN,16.0,8.0,15.0,...,0.6,**,4-4,33.8-35.6,35.8,420(-2),NaN,NaN,クリノゴッホ,52.0


In [30]:
df.isna().sum()

horse_id          0
日付                0
開催                0
天気               11
R                 0
レース名              0
映像          2439013
頭数                0
枠番                0
馬番                0
オッズ          179304
人気                9
着順                0
騎手                0
斤量                0
距離                0
馬場               11
馬場指数         969415
タイム               0
着差               11
ﾀｲﾑ指数        969914
通過            54865
ペース          754633
上り           295229
馬体重               0
厩舎ｺﾒﾝﾄ      2439013
備考          2439013
勝ち馬(2着馬)          5
賞金          1349748
dtype: int64

# 着順、頭数のデータ型変更

In [31]:
df['着順'] = df['着順'].str.strip('(降)')
df['着順'] = df['着順'].str.strip('(再')
df['着順'] = df['着順'].astype(float).astype(int)
df['着順'].unique()

array([ 3,  1,  2,  5,  4,  7,  8,  6, 10, 11,  9, 14, 12, 13, 15, 16, 17,
       18])

In [32]:
df['頭数'] = df['頭数'].astype(int)
df['頭数'].unique()

array([10, 14,  8, 16, 12,  9, 15,  5, 13, 11, 17,  7,  6, 18,  4,  3])

In [33]:
df['着順/頭数'] = df['着順'] / df['頭数']
df.head()

,horse_id,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,...,ﾀｲﾑ指数,通過,ペース,上り,馬体重,厩舎ｺﾒﾝﾄ,備考,勝ち馬(2着馬),賞金,着順/頭数
0,2020105571,2022/09/24,4中山6,曇,9.0,カンナS(OP),NaN,10,3.0,3.0,...,**,3-3,33.8-35.6,35.6,466(+2),NaN,NaN,ウメムスビ,403.3,0.300000
1,2020105571,2022/09/03,2札幌7,晴,1.0,2歳未勝利,NaN,14,7.0,12.0,...,**,1-1,35.0-35.7,35.7,464(-2),NaN,NaN,(コパカバーナビーチ),520.0,0.071429
2,2020105571,2022/08/27,2札幌5,曇,2.0,2歳未勝利,NaN,8,4.0,4.0,...,**,3-3,34.5-35.9,35.5,466(+2),NaN,NaN,バロットボックス,210.0,0.250000
3,2020105571,2022/08/06,1札幌5,晴,5.0,2歳新馬,NaN,8,3.0,3.0,...,**,5-5,36.1-34.0,34.4,464(0),NaN,NaN,メイクアスナッチ,70.0,0.625000
4,2020101863,2022/09/25,4中山7,晴,2.0,2歳未勝利,NaN,16,8.0,15.0,...,**,4-4,33.8-35.6,35.8,420(-2),NaN,NaN,クリノゴッホ,52.0,0.312500


# 頭数<=5は除外

In [34]:
df = df[df['頭数']>5]

# 学習用にデータ整形

In [35]:
temp_df = df.groupby('horse_id')
ids_per_group = temp_df.grouper.group_info[0]

In [36]:
df['ids_per_group'] = ids_per_group

In [37]:
count = 0
count_per_ids = []
for ids, v in enumerate(ids_per_group):
    if ids == 0 or ids_per_group[ids] != ids_per_group[ids-1]:
        count = 0
    else:
        count += 1
    count_per_ids += [count]

In [38]:
df['count_per_group'] = count_per_ids

In [39]:
pivot_df = df.pivot_table(values=['着順/頭数'], index=['horse_id'], columns=['count_per_group'], aggfunc='max')
pivot_df = pivot_df.iloc[:, :11]
pivot_df.reset_index(inplace=True)
pivot_df.columns = ['horse_id', '1試合前', '2試合前', '3試合前', '4試合前', '5試合前', '6試合前', '7試合前', '8試合前', '9試合前', '10試合前', '11試合前']
pivot_df = pivot_df.drop(columns=['1試合前'])

In [40]:
target_df = df.pivot_table(values=['着順/頭数'], index=['horse_id'], columns=['count_per_group'], aggfunc='max') 
target_df = target_df.iloc[:, :1]
target_df.reset_index(inplace=True)
target_df.columns = ['horse_id', 'target']
target_df = target_df.drop(columns=['horse_id'])
target_df[target_df['target']<0.2].head()
target_df = target_df.where(target_df['target'] <= 0.17, 0)
target_df = target_df.where(target_df['target'] == 0, 1)

In [41]:
pivot_df['target'] = target_df
pivot_df.to_csv(output_path, index=False)

In [42]:
pivot_df.head()

,horse_id,2試合前,3試合前,4試合前,5試合前,6試合前,7試合前,8試合前,9試合前,10試合前,11試合前,target
0,1988105984,0.600000,0.333333,0.066667,0.538462,0.083333,0.071429,0.8125,0.363636,0.214286,0.083333,0.0
1,1991103531,0.461538,0.666667,0.375000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,1991190011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,1992100038,0.555556,0.636364,0.375000,0.333333,0.833333,0.722222,0.6875,0.866667,0.800000,0.545455,0.0
4,1992101546,0.285714,0.250000,0.250000,0.750000,0.333333,0.625000,0.7000,0.857143,1.000000,0.846154,0.0


In [43]:
print(pivot_df[pivot_df['target']==1].shape)
print(pivot_df[pivot_df['target']==0].shape)

(6598, 12)
(100245, 12)


In [44]:
temp = '/Users/eee/python/jra_ml/data/horse_id_202209_200001/table/horse_result.csv'
df = pd.read_csv(temp)

/var/folders/8c/ctv36bkx7pv_b47nj404bc480000gn/T/ipykernel_87236/2954228279.py:2: DtypeWarning: Columns (17,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(temp)


In [45]:
df.tail()

,horse_id,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,...,着差,ﾀｲﾑ指数,通過,ペース,上り,馬体重,厩舎ｺﾒﾝﾄ,備考,勝ち馬(2着馬),賞金
2570735,1994100530,1997/04/19,3京都1,晴,8.0,ムーニーバレーRC賞(500万下),NaN,14.0,6.0,10.0,...,-0.4,NaN,3-2,34.6-35.3,35.1,500(0),NaN,NaN,(テイエムトッキュー),974.3
2570736,1994100530,1997/04/06,2阪神6,雨,6.0,君子蘭賞(500万下),NaN,16.0,4.0,8.0,...,0.2,NaN,7-3-1,35.3-37.7,37.8,500(-12),NaN,NaN,プレミアムサンダー,394.2
2570737,1994100530,1997/03/15,1阪神7,曇,1.0,4歳未勝利,NaN,8.0,4.0,4.0,...,-0.9,NaN,4-5-5-3,37.6-38.2,37.8,512(+18),NaN,NaN,(テンザングラマー),500.0
2570738,1994100530,1996/12/21,5阪神7,晴,5.0,3歳新馬,NaN,16.0,7.0,14.0,...,0.4,NaN,4-2-2,35.8-36.7,36.9,494(-2),NaN,NaN,オースミジャイアン,89.0
2570739,1994100530,1996/11/30,5阪神1,晴,4.0,3歳新馬,NaN,13.0,8.0,12.0,...,1.7,NaN,4-3,35.0-38.4,39.0,496(0),NaN,NaN,キョウエイマーチ,150.0
